# 03_ 주가 관련 보조지표를 활용하여 등락 예측_카카오

In [1]:
# 기본적으로 필요한 라이브러리
import numpy as np
import pandas as pd


#시각화 라이브러리
import matplotlib.pyplot as plt 
import seaborn as sns

#datetime 형식으로 바꿔주는 라이브러리
from datetime import datetime

In [2]:
stock = pd.read_csv('naver.csv')
stock.head()

,Date,Open,High,Low,Close,Volume
0,2020-09-01,322500,329500,322000,324500,691864
1,2020-09-02,332500,335500,327000,332500,808064
2,2020-09-03,333500,339500,329500,339000,776547
3,2020-09-04,322500,335000,322000,328500,1174337
4,2020-09-07,325500,325500,313500,314500,1308247


In [3]:
stock = stock.set_index('Date') # date를 index로 설정
stock = stock.loc['2020-09-01' : '2021-06-30']

## 1.1 머신러닝 준비

In [4]:
# 종가 기준 fluctuation 추가
def up_down(x):
    if x >= 0:
        return 1
    else :
        return 0

stock['fluctuation'] = (stock['Close'].shift(-1)-stock['Close']).apply(up_down)
stock.head()

,Open,High,Low,Close,Volume,fluctuation
Date,,,,,,
2020-09-01,322500,329500,322000,324500,691864,1
2020-09-02,332500,335500,327000,332500,808064,1
2020-09-03,333500,339500,329500,339000,776547,0
2020-09-04,322500,335000,322000,328500,1174337,0
2020-09-07,325500,325500,313500,314500,1308247,0


In [5]:
stock_train = stock.iloc[:-1]
target = stock_train['fluctuation']
stock_train = stock_train.drop('fluctuation', axis = 1)

from sklearn.model_selection import train_test_split
train, test, train_target, test_target = train_test_split(
    stock_train, target, test_size=0.3, shuffle=False)

## 1.2 모델링

In [6]:
# Importing Classifier Modules
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.linear_model import LogisticRegression   # logistic
from sklearn.neighbors import KNeighborsClassifier   # knn
from sklearn.naive_bayes import GaussianNB   # naive
from lightgbm import LGBMClassifier   # lightGBM
from sklearn import datasets
from sklearn import metrics

In [7]:
# 모형 생성
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic}, {'name' : 'KNN', 'model' : knn},
          {'name' : 'lgb.LGBM', 'model' : lgb}, {'name' : 'NaiveBayes', 'model' : naive}]

for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
    print ('model name : {}'.format(m['name']))
    print (metrics.classification_report(test_target, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
    print('Confusion Matrix') 
    print (metrics.confusion_matrix (test_target, predicted))

    print ('Accuracy Score : {:.4f}\n'.format(metrics.accuracy_score(test_target, predicted)))

model name : Logistic
              precision    recall  f1-score   support

           0       0.44      0.55      0.49        29
           1       0.50      0.39      0.44        33

    accuracy                           0.47        62
   macro avg       0.47      0.47      0.47        62
weighted avg       0.47      0.47      0.46        62

Confusion Matrix
[[16 13]
 [20 13]]
Accuracy Score : 0.4677

model name : KNN
              precision    recall  f1-score   support

           0       0.48      0.52      0.50        29
           1       0.55      0.52      0.53        33

    accuracy                           0.52        62
   macro avg       0.52      0.52      0.52        62
weighted avg       0.52      0.52      0.52        62

Confusion Matrix
[[15 14]
 [16 17]]
Accuracy Score : 0.5161

model name : lgb.LGBM
              precision    recall  f1-score   support

           0       0.65      0.45      0.53        29
           1       0.62      0.79      0.69        33


In [8]:
def rate_of_return():
    df['percent'] = round((df.Close-df.Close.shift(1))/df.Close.shift(1)*100, 2)
    for i in range(len(df)-1):
        if (df.loc[i, 'predicted'] == 0):
            df.loc[i+1, 'percent'] = df.loc[i+1, 'percent']

In [9]:
for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)
    
    df = pd.concat([test.reset_index().drop('Date', axis=1), pd.DataFrame(predicted, columns = ['predicted'])], axis=1)
    
    rate_of_return()
    
    df.dropna(inplace = True)

    print('model name : {}'.format(m['name']))
    print('첫날을 제외한 거래일수 : {}'.format(len(df)))
    print('누적 수익률 : {}'.format(round(df['percent'].sum(), 2)))
    print('1일 평균 수익률 : {}\n'.format(round(df['percent'].sum()/(len(df)-1),2)))

model name : Logistic
첫날을 제외한 거래일수 : 61
누적 수익률 : 9.31
1일 평균 수익률 : 0.16

model name : KNN
첫날을 제외한 거래일수 : 61
누적 수익률 : 9.31
1일 평균 수익률 : 0.16

model name : lgb.LGBM
첫날을 제외한 거래일수 : 61
누적 수익률 : 9.31
1일 평균 수익률 : 0.16

model name : NaiveBayes
첫날을 제외한 거래일수 : 61
누적 수익률 : 9.31
1일 평균 수익률 : 0.16



## 1.3 주가 관련 보조지표를 활용하여 등락 예측

In [10]:
# StockInsider 패키지를 사용해 여러가지 주가 관련 보조지표 산출
# 하지만 중국에서 개발된 패키지라 한국에 상장된 종목 코드는 등록되어 있지 않다.
# from_external_csv_data 함수를 이용해 코스피 종목의 주식데이터를 받아서 StockInsider 패키지에 넣어줘야 한다.

def get_stock(code):
    import FinanceDataReader as fdr
    from insider import StockInsider
    import os

    df=fdr.DataReader("code")
    df["day"]=df.index
    df.columns=['open', 'close','high', 'low', 'volumn', "price_change",'day']

    df["ma5"]=df["close"].rolling(5).mean()
    df["ma10"]=df["close"].rolling(10).mean()
    df["ma20"]=df["close"].rolling(20).mean()
    df["v_ma5"]=df["volumn"].rolling(5).mean()
    df["v_ma10"]=df["volumn"].rolling(10).mean()
    df["v_ma20"]=df["volumn"].rolling(20).mean()
    df["percent_change"]=df["close"].pct_change()

    
    if os.path.exists("./data")==False:

        os.mkdir("./data")

    df.to_csv("./data/{}.csv".format(code))
    fpath = "./data/{}.csv".format(code)
    si = StockInsider.from_external_csv_data(fpath=fpath, code=code)

    return si
    
# 주사 데이터는 FinanceDataReader 패키지로 가져오고, 전처리를 후에 csv파일로 저장한다.
# 이 파일을 이용해서 StockInsider 객체를 만든다

In [11]:
import FinanceDataReader as fdr
si=get_stock("035420")
si.plot()

In [12]:
# 이동평균선
def ma(df):
    #이동평균 칼럼 추가 당일 포함 3일 7일 15일 30일
    df['MA_3'] = df['Close'].rolling(window=3).mean() 
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_15'] = df['Close'].rolling(window=15).mean() 
    df['MA_30'] = df['Close'].rolling(window=30).mean()

In [13]:
# 지수이동평균선
def ema(df, day):
    df['EWM_{}'.format(day)] = df['Close'].ewm(span=day).mean()
    df['EWM_{}'.format(day)] = df['Close'].ewm(span=day).mean()

In [14]:
# 이격도 (PPO : Percentage Price oscillator)
def ppo(df, day):
    df['PPO_{}'.format(day)] = (df['Close']/df['MA_{}'.format(day)])*100

In [15]:
# RSI
def U(x):
    if x >= 0:
        return x
    else :
        return 0
    
def D(x):
    if x <= 0:
        return x*(-1)
    else :
        return 0

def rsi(df):
    df['diff_rsi'] = (df['Close'].shift(1)-df['Close'])
    df['AU'] = df['diff_rsi'].apply(U).rolling(window=14).mean() 
    df['AD'] = df['diff_rsi'].apply(D).rolling(window=14).mean() 
    df['RSI'] = df['AU']/(df['AU']+df['AD'])

In [16]:
# rsi(stock)
# stock.drop(['diff_rsi', 'AU', 'AD'], axis=1, inplace = True)

In [17]:
# 모멘텀 스토캐스틱 %K, %D, Fast, Slow
def high_low(day):
    global stock
    stock = stock.reset_index()
    for i in range(len(stock)-day+1):
        stock.loc[i, 'High_st']= stock[i:i+day]['High'].max()
        stock.loc[i, 'Low_st']= stock[i:i+day]['Low'].min()
        
    stock['High_st_4'] = stock['High_st'].shift(4)
    stock['Low_st_4'] = stock['Low_st'].shift(4)
    
    stock['fast_K'] = (stock['Close']-stock['Low_st_4'])/(stock['High_st_4']-stock['Low_st_4'])
    stock['fast_D'] = stock['fast_K'].rolling(3).mean()
    stock['slow_K'] = stock['fast_D']
    stock['slow_D'] = stock['slow_K'].rolling(3).mean()
    stock = stock.set_index('Date')

In [18]:
stock['High_st'] = np.nan
stock['Low_st'] = np.nan
high_low(5)

In [19]:
# CCI
def CCI(df):
    #CCI = (M-N) / (0.015*D)
    # M=특정일의 고가,저가, 종가의 평균
    # N = 일정기간동안의 단순이동평균 통상적으로 20일로 사용
    # D = M-N의 일정기간동안의 단순이동평균
    M = ((df.High)+(df.Low)+(df.Close)) / 3
    N = M.rolling(20).mean()
    D = (M-N).rolling(20).mean()
    CCI = (M - N)/ (0.015 * D)
    stock['CCI'] = CCI

In [20]:
stock = stock.dropna()

## 1.4 적정 보조지표 찾기

In [21]:
from sklearn import datasets
from sklearn import metrics

# 분류모형
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic}, {'name' : 'KNN', 'model' : knn},
          {'name' : 'lgb.LGBM', 'model' : lgb}, {'name' : 'NaiveBayes', 'model' : naive}]

In [22]:
while (True):
    index = []
    i = 0
    
    if i == 0:
        stock_train = stock.iloc[:-1]
        target = stock_train['fluctuation']
        stock_train = stock_train.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_train, target, test_size=0.3, shuffle=False)
        
        accuracy = []
        
        for m in models:
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        init_score = np.mean(accuracy)
        
        print(i, ' iteration, initial score : {}'.format(round(init_score, 4)))
        
        i += 1    
        
    if i == 1:
        stock_2 = stock.copy()
        
        index2 = [ma, ema, ppo]
        
        for dex in index2:
            if dex == ma:
                dex(stock_2)
            if dex == ema:
                dex(stock_2, 15)
                dex(stock_2, 30)
            if dex == ppo:
                dex(stock_2, 15)
                dex(stock_2, 30)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index = index2
            
        i += 1
        
    
    if i == 2:
        stock_2 = stock.copy()
        
        rsi(stock_2)
        stock_2.drop(['diff_rsi', 'AU', 'AD'], axis=1, inplace = True)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index.append(rsi)
        
        i += 1
        
    if i == 3:
        stock_2 = stock.copy()
        
        stock_2['High_st'] = np.nan
        stock_2['Low_st'] = np.nan
        high_low(5)
        stock_2 = stock_2.drop(['High_st', 'Low_st', 'High_st_4', 'Low_st_4', 'fast_K', 'fast_D'], axis = 1)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index.append(high_low)
           
        i += 1
        
        
    if i == 4:
        stock_2 = stock.copy()
        
        CCI(stock_2)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index.append(CCI)

        break

0  iteration, initial score : 0.5948
1  iteration, Accuracy score : 0.5938
2  iteration, Accuracy score : 0.6146
3  iteration, Accuracy score : 0.5417
4  iteration, Accuracy score : 0.6094


In [23]:
# 분류모형
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic}, {'name' : 'KNN', 'model' : knn},
          {'name' : 'lgb.LGBM', 'model' : lgb}, {'name' : 'NaiveBayes', 'model' : naive}]


for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
    print ('model name : {}'.format(m['name']))
    print (metrics.classification_report(test_target, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
    print('Confusion Matrix') 
    print (metrics.confusion_matrix (test_target, predicted))

    print ('Accuracy Score : {:.4f}\n'.format(metrics.accuracy_score(test_target, predicted)))

model name : Logistic
              precision    recall  f1-score   support

           0       0.70      0.83      0.76        23
           1       0.81      0.68      0.74        25

    accuracy                           0.75        48
   macro avg       0.76      0.75      0.75        48
weighted avg       0.76      0.75      0.75        48

Confusion Matrix
[[19  4]
 [ 8 17]]
Accuracy Score : 0.7500

model name : KNN
              precision    recall  f1-score   support

           0       0.67      0.26      0.38        23
           1       0.56      0.88      0.69        25

    accuracy                           0.58        48
   macro avg       0.62      0.57      0.53        48
weighted avg       0.61      0.58      0.54        48

Confusion Matrix
[[ 6 17]
 [ 3 22]]
Accuracy Score : 0.5833

model name : lgb.LGBM
              precision    recall  f1-score   support

           0       0.53      0.43      0.48        23
           1       0.55      0.64      0.59        25


In [24]:
for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)
    
    df = pd.concat([test.reset_index().drop('Date', axis=1), pd.DataFrame(predicted, columns = ['predicted'])], axis=1)
    
    rate_of_return()
    
    df.dropna(inplace = True)

    print('model name : {}'.format(m['name']))
    print('첫날을 제외한 거래일수 : {}'.format(len(df)))
    print('누적 수익률 : {}'.format(round(df['percent'].sum(), 2)))
    print('1일 평균 수익률 : {}\n'.format(round(df['percent'].sum()/(len(df)-1),2)))

model name : Logistic
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19

model name : KNN
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19

model name : lgb.LGBM
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19

model name : NaiveBayes
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19



## 2.1 감성분석 데이터를 포함하여 등락 예측

In [25]:
score = pd.read_csv('네이버_감성분석.csv')

In [26]:
score_3 = score.copy()
stock_3 = stock_2.copy()

In [27]:
# stock과 score의 갯수가 달라 맞는 date를 찾아 출력하는 함수
def date_range():
    score_num = len(score_3)
    for i in range(1, 30):
        date_num = len(pd.Date_range('9/{}/2020'.format(i), '6/30/2021'))
        if score_num == date_num:
            date = pd.Date_range('9/{}/2020'.format(i), '6/30/2021')
            break
    return date

In [28]:
stock_3 = stock_3.reset_index() # 합치기 위해 날짜 인덱스를 열로 보냄

#stock_3데이터와 score_3데이터를 한 dataframe으로 합치기 위함
stock_data = pd.merge(stock_3, score_3, how='left')
stock_data.dropna(inplace=True)
stock_data = stock_data.set_index('Date') # index를 date로 설정

In [29]:
stock_data.head()

,Open,High,Low,Close,Volume,fluctuation,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,
2020-11-03,284000,287000,280000,283000,1066923,1,307000.0,280000.0,301000.0,280000.0,0.142857,0.283391,0.283391,0.530778,1,0,0
2020-11-04,282500,300500,282000,298500,1623712,1,307000.0,282000.0,301000.0,280000.0,0.880952,0.422571,0.422571,0.411472,1,0,0
2020-11-05,295000,302000,291500,299000,1082388,1,307000.0,277500.0,302000.0,280000.0,0.863636,0.629149,0.629149,0.445037,0,1,0
2020-11-06,303000,307000,299000,301000,968971,0,307000.0,277500.0,307000.0,280000.0,0.777778,0.840789,0.840789,0.630836,1,0,0
2020-11-09,304000,304500,298000,298000,678531,0,304500.0,277500.0,307000.0,280000.0,0.666667,0.769360,0.769360,0.746433,1,0,0


In [30]:
len(stock_data)

160

## 2.2 머신러닝 준비

In [31]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler() 
# columns = [i for i in stock_data.columns]
# stock_data[columns] = scaler.fit_transform(stock_data)

In [32]:
from sklearn.model_selection import train_test_split

stock_df = stock_data.iloc[:-1]
target = stock_df['fluctuation']
stock_df = stock_df.drop('fluctuation', axis = 1)

train, test, train_target, test_target = train_test_split(
    stock_df, target, test_size = 0.3, shuffle=False )

In [33]:
train.head()

,Open,High,Low,Close,Volume,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,
2020-11-03,284000,287000,280000,283000,1066923,307000.0,280000.0,301000.0,280000.0,0.142857,0.283391,0.283391,0.530778,1,0,0
2020-11-04,282500,300500,282000,298500,1623712,307000.0,282000.0,301000.0,280000.0,0.880952,0.422571,0.422571,0.411472,1,0,0
2020-11-05,295000,302000,291500,299000,1082388,307000.0,277500.0,302000.0,280000.0,0.863636,0.629149,0.629149,0.445037,0,1,0
2020-11-06,303000,307000,299000,301000,968971,307000.0,277500.0,307000.0,280000.0,0.777778,0.840789,0.840789,0.630836,1,0,0
2020-11-09,304000,304500,298000,298000,678531,304500.0,277500.0,307000.0,280000.0,0.666667,0.769360,0.769360,0.746433,1,0,0


In [34]:
test.tail()

,Open,High,Low,Close,Volume,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,
2021-06-17,389500,394500,385000,389500,670522,427000.0,385000.0,395000.0,367500.0,0.800000,0.871674,0.871674,0.890845,1,0,0
2021-06-18,391500,400500,391000,398000,1268656,436000.0,391000.0,400500.0,375500.0,0.900000,0.860199,0.860199,0.886305,1,0,0
2021-06-21,398000,402000,392000,397000,809114,436000.0,391000.0,402000.0,381500.0,0.756098,0.818699,0.818699,0.850191,1,0,0
2021-06-22,400000,400500,391000,391000,702840,436000.0,391000.0,402000.0,383000.0,0.421053,0.692383,0.692383,0.790427,1,0,0
2021-06-23,393000,427000,392500,423500,2721355,436000.0,392500.0,427000.0,385000.0,0.916667,0.697939,0.697939,0.736341,1,0,0


## 2.3 모델링

In [35]:
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic},
        {'name' : 'KNN', 'model' : knn},
        {'name' : 'lgb.LGBM', 'model' : lgb},
        {'name' : 'NaiveBayes', 'model' : naive}]

for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
    print ('model name : {}'.format(m['name']))
    print (metrics.classification_report(test_target, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
    print('Confusion Matrix') 
    print (metrics.confusion_matrix (test_target, predicted))

    print ('Accuracy Score : {:.4f}\n'.format(metrics.accuracy_score(test_target, predicted)))

model name : Logistic
              precision    recall  f1-score   support

           0       0.68      0.83      0.75        23
           1       0.80      0.64      0.71        25

    accuracy                           0.73        48
   macro avg       0.74      0.73      0.73        48
weighted avg       0.74      0.73      0.73        48

Confusion Matrix
[[19  4]
 [ 9 16]]
Accuracy Score : 0.7292

model name : KNN
              precision    recall  f1-score   support

           0       0.67      0.26      0.38        23
           1       0.56      0.88      0.69        25

    accuracy                           0.58        48
   macro avg       0.62      0.57      0.53        48
weighted avg       0.61      0.58      0.54        48

Confusion Matrix
[[ 6 17]
 [ 3 22]]
Accuracy Score : 0.5833

model name : lgb.LGBM
              precision    recall  f1-score   support

           0       0.53      0.43      0.48        23
           1       0.55      0.64      0.59        25


In [36]:
for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)
    
    df = pd.concat([test.reset_index().drop('Date', axis=1),
                    pd.DataFrame(predicted, columns = ['predicted'])], axis=1)
    
    rate_of_return()
    
    df.dropna(inplace = True)

    print('model name : {}'.format(m['name']))
    print('첫날을 제외한 거래일수 : {}'.format(len(df)))
    print('누적 수익률 : {}'.format(round(df['percent'].sum(), 2)))
    print('1일 평균 수익률 : {}\n'.format(round(df['percent'].sum()/(len(df)-1),2)))

model name : Logistic
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19

model name : KNN
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19

model name : lgb.LGBM
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19

model name : NaiveBayes
첫날을 제외한 거래일수 : 47
누적 수익률 : 8.84
1일 평균 수익률 : 0.19



In [37]:
stock_data.head()

,Open,High,Low,Close,Volume,fluctuation,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,
2020-11-03,284000,287000,280000,283000,1066923,1,307000.0,280000.0,301000.0,280000.0,0.142857,0.283391,0.283391,0.530778,1,0,0
2020-11-04,282500,300500,282000,298500,1623712,1,307000.0,282000.0,301000.0,280000.0,0.880952,0.422571,0.422571,0.411472,1,0,0
2020-11-05,295000,302000,291500,299000,1082388,1,307000.0,277500.0,302000.0,280000.0,0.863636,0.629149,0.629149,0.445037,0,1,0
2020-11-06,303000,307000,299000,301000,968971,0,307000.0,277500.0,307000.0,280000.0,0.777778,0.840789,0.840789,0.630836,1,0,0
2020-11-09,304000,304500,298000,298000,678531,0,304500.0,277500.0,307000.0,280000.0,0.666667,0.769360,0.769360,0.746433,1,0,0


In [38]:
stock_data.head()

,Open,High,Low,Close,Volume,fluctuation,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,
2020-11-03,284000,287000,280000,283000,1066923,1,307000.0,280000.0,301000.0,280000.0,0.142857,0.283391,0.283391,0.530778,1,0,0
2020-11-04,282500,300500,282000,298500,1623712,1,307000.0,282000.0,301000.0,280000.0,0.880952,0.422571,0.422571,0.411472,1,0,0
2020-11-05,295000,302000,291500,299000,1082388,1,307000.0,277500.0,302000.0,280000.0,0.863636,0.629149,0.629149,0.445037,0,1,0
2020-11-06,303000,307000,299000,301000,968971,0,307000.0,277500.0,307000.0,280000.0,0.777778,0.840789,0.840789,0.630836,1,0,0
2020-11-09,304000,304500,298000,298000,678531,0,304500.0,277500.0,307000.0,280000.0,0.666667,0.769360,0.769360,0.746433,1,0,0
